**Modulo MSI** 
*Input = .docx + .xlsx*


In [1]:
from dotenv import load_dotenv
from docx_parser import getDoc
from docx_parser import getCsv
from langchain import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
import os

In [2]:
# Cargo variables de configuración
load_dotenv()

True

In [3]:
# Carga el documento con los casos de prueba 7500 palabras aproximadamente.
#context_file='documentos/migraciones/cp_migraciones.docx'
#context_file='documentos/redenciones/cp_redenciones.docx'
context_file = 'documentos/msi/cp_msi.docx'

# Convierte el .docx a string
document = getDoc(context_file)

In [4]:
# Carga el documento con los casos de prueba 7500 palabras aproximadamente.
#context_file1='documentos/migraciones/-------no tiene HU migraciones'
#context_file1='documentos/redenciones/hu_redenciones.docx'
context_file1='documentos/msi/hu_msi.docx'

# Convierte el .docx a string.
document1=getDoc(context_file1)

In [5]:
# Carga csv generado a partir del excel de input.
# La tabla fue reducida en columnas por límite de tokens.
#csv_file_path = "documentos/migraciones/cp_migraciones.csv"
#csv_file_path = "documentos/redenciones/cp_redenciones.csv"
csv_file_path = "documentos/msi/cp_msi.csv"

# Convierte a string
tabla = getCsv(csv_file_path)

In [6]:
# Define plantilla de prompt, pide CSV.
template = """
Sos parte del equipo de testing re una compania de telecomunicaciones. Te voy a dar dos documentos (un texto y una tabla) con los requerimientos para testeo de uno de los modulos de una aplicacion.
 Necesito que identifiques todos los casos de prueba que puedas identificar. Deberás incluir la condicion esperada para cada uno de los casos.
 Dame el resultado en una tabla formato csv con las columnas 'CasoDePrueba' y 'ResultadoEsperado'. La columna 'Caso de prueba' debe contener el nombre del caso de prueba.
   Esta es la tabla {tabla}
   Este es el texto: {document}
"""
# Crea prompt como un objeto de Langchain.
prompt = PromptTemplate.from_template(template)

In [7]:
# Deefine plantilla prompt.
template = """
Sos parte del equipo de testing re una compania de telecomunicaciones. Te voy a dar dos documentos (un texto y una tabla) con los requerimientos para testeo de uno de los modulos de una aplicacion.
 Necesito que identifiques todos los casos de prueba que puedas identificar. Deberás incluir la condicion esperada para cada uno de los casos.
   Esta es la tabla {tabla}
   Este es el texto: {document}
"""
# Crea prompt como un objeto de Langchain.
prompt = PromptTemplate.from_template(template)

In [8]:

# Define LLM de back.
llm = AzureChatOpenAI (
        azure_endpoint=os.getenv('API_BASE'),
        openai_api_version=os.getenv('API_VERSION'),
        deployment_name=os.getenv('CHAT_ENGINE_16K'),
        openai_api_key=os.getenv('API_KEY'),
        openai_api_type=os.getenv('API_TYPE'),
        temperature=0,
        model_version="0613" # es importante aclararlo para el correcto calculo de los costos
        )

# Crea la cadena de Langchain.
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Ejecuta la cadena con los documentos de input insertados en el prompt.
result = llm_chain.run({"document":document, "tabla": tabla})

In [9]:
print(result)

Casos de prueba identificados:

1. Caso de prueba: Compra menor a $3,000
   - Descripción: Se ingresa a la aplicación y muestra el banner para adquirir una terminal. Se selecciona una terminal con monto menor a $3,000 y la aplicación no muestra las opciones de MSI.
   - DN: 5537685234
   - Área Funcional / Sub proceso: Venta de Terminales
   - Funcionalidad / Característica: MSI
   - Datos / Acciones de Entrada: 
     * Banner de Venta Terminales
     * Selección de terminal
     * Aceptar términos y condiciones
     * Validar opciones de pasarela de pago
   - Resultado Esperado: La aplicación no muestra las opciones para MSI desde la pasarela de pago.
   - Resultado Obtenido: Se realizaron pruebas con éxito
   - Observaciones: Ok

2. Caso de prueba: Compra superior a $3,000
   - Descripción: Se ingresa a la aplicación y muestra el banner para adquirir una terminal. Se selecciona una terminal con monto superior a $3,000 y la aplicación muestra las opciones de MSI.
   - DN: 5537685234
 

In [10]:
# Guardo los datos en un csv
with open('testing_oneshot_docxls.csv', 'w') as file:
    file.write(result)

*Prompt en ingles*

In [9]:
# Define plantill prompt.
template_eng = """
You are part of the testing team for a telecommunications company. I'm going to provide you with two documents (a text and a table) with the testing requirements for one of the modules of an application.
I need you to identify all the test cases you can find. You should include the expected condition for each case.
Please provide the result in a CSV format table with columns 'CasoDePrueba' and 'ResultadoEsperado'. The 'CasoDePrueba' column should contain the name of each test case. The resultmust be in spanish.
This is the table: {tabla}
This is the text: {document}
"""
# Crea prompt como un objeto de Langchain.
prompt_eng = PromptTemplate.from_template(template_eng)

In [10]:
# Crea la cadena de Langchain.
llm_chain = LLMChain(llm=llm, prompt=prompt_eng)
# Ejecuta la cadena con los documentos de input insertados en el prompt.
result=llm_chain.run({"document":document, "tabla": tabla})

In [11]:
print(result)

CasoDePrueba;ResultadoEsperado
Compra menor a $3,000;La aplicación no muestra las opciones para MSI desde la pasarela de pago.
Compra superior a $3,000;La aplicación muestra las opciones para MSI desde la pasarela de pago.
